# Project Goal

The goal of this project is to **analyze customer ordering behavior and identify factors that contribute to cart abandonment** within the **MyCoke360** platform — Coca-Cola’s online ordering system for business customers.

MyCoke360 allows Coca-Cola’s B2B clients — including restaurants, retailers, distributors, and institutions — to place and manage orders digitally.  
Despite high engagement on the platform, many customers begin the purchase process but fail to complete their orders.

This analysis aims to:
- Quantify **cart abandonment rates**, and  
- Uncover **behavioral and operational drivers** behind this issue.

By linking **Google Analytics (GA)** event data (user interactions) with **order**, **visit**, and **sales** records, we seek to generate actionable insights that will help:
- Reduce lost sales opportunities, and  
- Improve the overall **conversion efficiency** of the MyCoke360 platform.

---

# Purpose of This EDA Notebook

This **Exploratory Data Analysis (EDA)** notebook serves as the **data foundation** for the MyCoke360 Cart Abandonment Study.

The objectives of this notebook are to:

- **Explore and clean** all relevant datasets from MyCoke360’s data warehouse  
- **Validate data quality** and ensure consistent linking across systems  
- **Construct unified customer-level order windows** using visit frequency and cutoff times  
- **Explore behavioral and operational trends** in ordering and cart activity  
- **Identify potential drivers and hypotheses** that explain cart abandonment behavior  

The outcomes of this EDA will inform future:
- **Predictive modeling**,  
- **Customer segmentation**, and  
- **Strategic business recommendations**  

to enhance MyCoke360’s digital customer experience and **increase order conversion rates**.

---

# Data Overview

This project integrates multiple data sources from **Swire Coca-Cola’s MyCoke360** platform to study **cart abandonment behavior** among business customers.  
The dataset spans from **May 31, 2024, to May 26, 2025**, covering one full year of platform activity.

Our analysis combines **behavioral**, **transactional**, and **operational** data to understand how customers interact with the MyCoke360 system and identify when they add products to their carts but fail to complete purchases.

---

## Purpose of Data Integration

Each dataset provides a different view of customer behavior and operations:

| **Dataset** | **Description** | **Purpose in Analysis** |
|--------------|----------------|-------------------------|
| **Google Analytics (GA)** | Logs user interactions on MyCoke360 — including page views, button clicks, add/remove cart actions, and purchases. | Identifies **cart activity** and measures engagement and abandonment within order windows. |
| **Orders** | Contains actual orders placed by customers, including timestamps, quantities, and materials. | Verifies whether a purchase occurred within the defined **order window** and corrects for missing GA purchase events. |
| **Sales** | Captures financial details such as `NSI_DEAD_NET`, `PHYSICAL_VOLUME`, and `BOTTLER_PROFIT`. | Used to estimate **revenue loss** from abandoned carts and compute average order values. |
| **Visit Plan History** | Historical schedule of each customer’s delivery policy (anchor dates and frequency of orders). | Defines the expected **order windows** using anchor dates, frequencies, and cutoff times. |
| **Operating Hours** | Current delivery frequency and anchor day for each customer. | Validates the latest delivery policy and helps link customers to visit plan data. |
| **Cutoff Times** | Specifies order cutoff times for each plant and distribution mode. | Used to calculate **ANCHOR_CUTOFF** and **NEXT_ANCHOR_CUTOFF** per customer. |
| **Customer** | Customer profile data including sales office, distribution mode, and cold drink channel. | Enables **segmentation** by region, delivery type, and customer category. |
| **Material** | Product catalog with brand, pack type, flavor, and beverage category. | Helps identify **which product types** are more likely to be abandoned. |

---

## Key Data Characteristics

- **Date Range:** May 31, 2024 – May 26, 2025  
- **Customer Type:** B2B (Food Service On Premise) customers  
- **Geographic Scope:** U.S. sales offices (e.g., Draper, Tempe, Denver, Wilsonville, Bellevue)  
- **Unit of Analysis:** Customer × Order Window  

---

## Data Volumes

| **Dataset** | **File Name** | **Rows** | **Columns** |
|--------------|---------------|-----------|--------------|
| Google Analytics | `google_analytics.csv` | 3,704,088 | 10 |
| Orders | `orders.csv` | 1,662,157 | 7 |
| Sales | `sales.csv` | 499,787 | 8 |
| Visit Plan History | `visit_plan.csv` | 14,796,017 | 9 |
| Customer | `customer.csv` | 6,334 | 7 |
| Cutoff Times | `cutoff_times.csv` | 220 | 5 |
| Operating Hours | `operating_hours.csv` | 6,202 | 4 |
| Material | `material.csv` | 1,252 | 6 |

---

## Data Cleaning and Preprocessing Steps

The following preprocessing and validation steps were performed during EDA:

### **Customer Table**
- Dropped test or inactive accounts.  
- Standardized column names (`CUSTOMER_NUMBER` → `CUSTOMER_ID`).  
- Cleaned `DISTRIBUTION_MODE_DESCRIPTION` (replaced nulls with "OFS").  

### **Cutoff Times**
- Dropped invalid rows with missing or zero-valued `SALES_OFFICE_DESCRIPTION`.  
- Filled missing cutoff times with the default 5:00 PM value.  

### **Visit Plan**
- Removed nulls (<1% of records).  
- Standardized frequency codes (`01` = weekly, `02` = biweekly, `04` = monthly).  
- Validated `ANCHOR_DATE` and created calculated fields:  
  ```python
  ANCHOR_CUTOFF = ANCHOR_DATE + CUTOFF_TIME  
  NEXT_ANCHOR_CUTOFF = ANCHOR_DATE + FREQUENCY + CUTOFF_TIME

## Customer Table — Data Import and Initial Inspection

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
customer = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/customer.csv")
print(customer.head())
print("*****\n")

# Check for missing values in each column
print("Missing values per column:")
print(customer.isnull().sum())
print("*****\n")

print("Value counts for 'DISTRIBUTION_MODE_DESCRIPTION':")
print(customer['DISTRIBUTION_MODE_DESCRIPTION'].value_counts(dropna=False))
print("*****\n")

customer_clean = customer.copy()
# Fill missing values in 'DISTRIBUTION_MODE_DESCRIPTION' with 'OFS'
customer_clean['DISTRIBUTION_MODE_DESCRIPTION'] = customer_clean['DISTRIBUTION_MODE_DESCRIPTION'].fillna('OFS')
# Rename 'CUSTOMER_NUMBER' to 'CUSTOMER_ID' 
customer_clean = customer_clean.rename(columns={'CUSTOMER_NUMBER': 'CUSTOMER_ID'})
# Display the first few rows of the cleaned dataset
print("*****\n")
print("First 5 rows of the cleaned customer dataset (customer_clean):")
print(customer_clean.head())

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/customer.csv'

In [ ]:
# a graph that shows count of customers by distribution mode
customer_clean.groupby('DISTRIBUTION_MODE_DESCRIPTION')['CUSTOMER_ID'].count()\
.sort_values(ascending=False)\
.plot(kind='bar')

## Cutoff Times Table — Data Import and Cleaning

In [ ]:
cutoff_times = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/cutoff_times.csv")

# Check for missing values
print("Missing values in cutoff_times:")
print(cutoff_times.isnull().sum())
print("*****\n")

# Inspect unique values in DISTRIBUTION_MODE
print("Distribution of 'DISTRIBUTION_MODE':")
print(cutoff_times['DISTRIBUTION_MODE'].value_counts(dropna=False))
print("*****\n")

# Rename columns for consistency with other tables
cutoff_times = cutoff_times.rename(columns={
    'DISTRIBUTION_MODE': 'DISTRIBUTION_MODE_DESC',
    'SALES_OFFICE': 'SALES_OFFICE_DESCRIPTION'
})

# Check distribution of SALES_OFFICE_DESCRIPTION
print("Value counts for 'SALES_OFFICE_DESCRIPTION':")
sales_office_counts = cutoff_times['SALES_OFFICE_DESCRIPTION'].value_counts()
print(sales_office_counts)
print("*****\n")

# Remove 4 invalid rows where SALES_OFFICE_DESCRIPTION == '0'
print("Removing rows where SALES_OFFICE_DESCRIPTION == '0' (invalid entries)...")
cutoff_clean = cutoff_times[cutoff_times['SALES_OFFICE_DESCRIPTION'] != '0']


cutoff_clean.rename(columns = {'CUTOFFTIME__C':'CUTOFF_TIME'}, inplace= True)
cutoff_clean["CUTOFF_TIME"].fillna("5:00:00 PM")
print(cutoff_clean.head())


## Material Table — Data Import and Inspection

In [ ]:
material = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/material.csv")
print(material.head())
print("*****\n")

# Check for missing values
print("Missing values per column:")
print(material.isnull().sum())

print("Value counts for 'BEV_CAT_DESC':")
print(material['BEV_CAT_DESC'].value_counts(dropna=False))

## Operating Hours Table — Data Import and Inspection

In [ ]:
operating_hours = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/operating_hours.csv")
print(operating_hours.head())
print("*****\n")

# Check for missing values
print("Missing values per column:")
null_counts = operating_hours.isnull().sum()
print(null_counts)

# Inspect original 'FREQUENCY' values
print("Original 'FREQUENCY' value counts:")
print(operating_hours['FREQUENCY'].value_counts(dropna=False))
print("*****\n")
# Map frequency descriptions to standardized codes
map_frequency = {
    'Every Week': '01',
    'Every 2 Weeks': '02',
    'Every 3 Weeks': '03',
    'Every 4 Weeks': '04'
}
operating_hours['FREQUENCY_CLEAN'] = operating_hours['FREQUENCY'].map(map_frequency)
print(operating_hours['FREQUENCY_CLEAN'].value_counts(dropna=False))
print("\nFrequency mapping complete.")

## Orders Table — Data Cleaning and Preparation

In [ ]:
orders = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/orders.csv")
print(orders.head())
print("******************\n")
# Check for missing values
null_counts = orders.isnull().sum()
print(null_counts)

print(f"\n Total rows: {len(orders):,}")
print(f"Missing MATERIAL_ID: {null_counts['MATERIAL_ID']}, Missing PLANT_ID: {null_counts['PLANT_ID']}")
print("******************\n")

# Remove rows with any missing values (especially critical for IDs)
print("Dropping rows with missing IDs")
orders_clean = orders.dropna().copy()

# Convert date columns to datetime
print("Converting 'CREATED_DATE_EST' and 'CREATED_DATE_UTC' to datetime")
orders_clean['CREATED_DATE_EST'] = pd.to_datetime(orders_clean['CREATED_DATE_EST'])
orders_clean['CREATED_DATE_UTC'] = pd.to_datetime(orders_clean['CREATED_DATE_UTC'])

# Display date range for CREATED_DATE_UTC
min_date = orders_clean['CREATED_DATE_UTC'].min()
max_date = orders_clean['CREATED_DATE_UTC'].max()
print(f"📅 Date range (UTC): {min_date} to {max_date}")
print("******************\n")

# Check for outliers in ORDER_QUANTITY
print("Descriptive statistics for 'ORDER_QUANTITY':")
order_qty_stats = orders_clean['ORDER_QUANTITY'].describe()
print(order_qty_stats)



## Sales Table — Data Import and Exploration

In [ ]:
sales = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/sales.csv")
print(sales.head())
print("******************\n")
# check for nulls
print(sales.isnull().sum())
# 5 highest values for nsi_dead_net
print(sales.nlargest(5, 'NSI_DEAD_NET'))
print("******************\n")


## Visit Plan Table — Data Import, Cleaning, and Frequency Standardization

In [ ]:
visit_plan = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/visit_plan.csv")
print(visit_plan.head())
print("******************\n")

# check for nulls
print("Missing values per column:")
null_counts = visit_plan.isnull().sum()
total_rows = len(visit_plan)
print(null_counts)
print("******************\n")

# Map DISTRIBUTION_MODE codes to descriptions
visit_plan['DISTRIBUTION_MODE'].value_counts()
code_to_description = {
    'OF': 'OFS',
    'SL': 'Sideload',
    'BK': 'Bulk Distribution',
    'FS': 'Full Service',
    'RD': 'Rapid Delivery',
    'NR': 'Night Rapid Delivery',
    'NS': 'Tell Sell',
    'NO': 'Night OFS',
    'EZ': 'E Pallet',
    'SE': 'Special Events',
    'DD': 'Night Sideload'
}
# creating distribution_mode_desc column
visit_plan['DISTRIBUTION_MODE_DESC'] = visit_plan['DISTRIBUTION_MODE'].map(code_to_description)
visit_plan['DISTRIBUTION_MODE_DESC'].value_counts()

In [ ]:
# Convert ANCHOR_DATE to datetime
visit_clean = visit_plan.copy()
visit_clean['ANCHOR_DATE'] = pd.to_datetime(visit_clean['ANCHOR_DATE'], errors='coerce')

# Add day of week
visit_clean['WEEK_DAY_OF_ANCHOR_DATE'] = visit_clean['ANCHOR_DATE'].dt.day_name()

# Clean and standardize FREQUENCY column
visit_clean['FREQUENCY'] = visit_clean['FREQUENCY'].astype(str).str.strip()

# Apply mapping to standardize frequency values
freq_mapping = {
    "01": 1, "1": 1, "1.0": 1, "1 ": 1, " 1": 1, "Every Week On": 1,
    "02": 2, "2": 2, "2.0": 2, "2 ": 2, "Every Second Week On": 2,
    "03": 3, "3": 3, "3.0": 3, "3 ": 3, "Every Third Week On": 3,
    "04": 4, "4": 4, "4.0": 4, "4 ": 4, "Every Fourth Week On": 4,
    "05": 5, "5": 5, "5.0": 5, "5 ": 5, "Every Fifth Week On": 5,
    "06": 6, "6": 6, "6.0": 6, "Every Sixth Week On": 6,
    "08": 8, "8": 8, "8.0": 8, "Every Eighth Week On": 8,
    "10": 10, "10.0": 10, "Every Tenth Week On": 10,
    "Not Applicable": None
}

# Map and convert to numeric
visit_clean['FREQUENCY_NUM'] = visit_clean['FREQUENCY'].replace(freq_mapping)
visit_clean['FREQUENCY_NUM'] = pd.to_numeric(visit_clean['FREQUENCY_NUM'], errors='coerce').astype('Int64')

# Convert to days (weeks * 7)
visit_clean['WINDOW_FREQUENCY'] = visit_clean['FREQUENCY_NUM'] * 7

## Google Analytics (GA) Data — Event-Level Cleaning and Integration

In [ ]:
ga = pd.read_csv("C:/Users/91744/Desktop/MSBA/sem 4/capstone/csv_files/google_analytics.csv")
print(ga.head())
print("******************\n")

# Check for missing values
null_counts = ga.isnull().sum()
print(null_counts)

# Show non-null counts for context
print("Non-null counts per column:")
print(ga.count())
print("******************\n")

# Inspect EVENT_PAGE_TITLE vs EVENT_PAGE_NAME (often redundant)
print(ga[['EVENT_PAGE_TITLE', 'EVENT_PAGE_NAME']].head())
print("******************\n")

ga_clean = ga.copy()

# Convert EVENT_TIMESTAMP to datetime (UTC)
print("Converting 'EVENT_TIMESTAMP' to datetime (UTC)...")
ga_clean["EVENT_TIMESTAMP_UTC"] = (
    pd.to_datetime(ga_clean["EVENT_TIMESTAMP"], errors="coerce")
    .dt.tz_localize(None) 
)
# Check how many timestamps failed conversion
invalid_timestamps = ga_clean["EVENT_TIMESTAMP_UTC"].isnull().sum()
if invalid_timestamps > 0:
    print(f"{invalid_timestamps:,} rows have invalid timestamps (set to NaT).")
else:
    print("All timestamps successfully parsed.")
print("******************\n")

# Handle missing values in DEVICE_MOBILE_BRAND_NAME
print("Top mobile brands (before cleaning):")
print(ga_clean['DEVICE_MOBILE_BRAND_NAME'].value_counts(dropna=False).head(10))
print("******************\n")

# Fill missing brand names with 'Unknown'
ga_clean['DEVICE_MOBILE_BRAND_NAME'] = ga_clean['DEVICE_MOBILE_BRAND_NAME'].fillna('Unknown')
print(ga_clean['DEVICE_MOBILE_BRAND_NAME'].value_counts().head(10))
print("******************\n")

# Drop redundant page columns
print("Dropping redundant columns: 'EVENT_PAGE_TITLE', 'EVENT_PAGE_NAME'")
ga_clean = ga_clean.drop(columns=['EVENT_PAGE_TITLE', 'EVENT_PAGE_NAME'])

In [ ]:
cutoff_clean.head()

In [ ]:
cutoff_clean.shape

In [ ]:
visit_w_cutoff = pd.merge(
    visit_clean,
    cutoff_times,
    left_on=["SALES_OFFICE","SHIPPING_CONDITIONS_DESC","DISTRIBUTION_MODE_DESC"],
    right_on=["PLANT_ID","SHIPPING_CONDITION_TIME","DISTRIBUTION_MODE_DESC"],
    how="left"
)
print(visit_w_cutoff.shape)

In [ ]:
visit_w_cutoff = visit_w_cutoff.drop(columns=["FREQUENCY", "ELT_TS","SNAPSHOT_DATE","SALES_OFFICE_DESC",
                              "SHIPPING_CONDITION_TIME","PLANT_ID"])
print(visit_w_cutoff.shape)

In [ ]:
visit_w_date = visit_w_cutoff.copy()
# Master table included data out of our needed range
visit_w_date['ANCHOR_DATE'] = pd.to_datetime(visit_w_date['ANCHOR_DATE'])
visit_w_date = visit_w_date[(visit_w_date['ANCHOR_DATE'] > pd.to_datetime('2024-06-05')) & 
       (visit_w_date['ANCHOR_DATE'] < pd.to_datetime('2025-05-26'))]


visit_w_date = visit_w_date.drop_duplicates(subset=['CUSTOMER_ID', 'ANCHOR_DATE']).sort_values('ANCHOR_DATE')
print(visit_w_date.shape)

In [ ]:
# Fill in between the unique ANCHOR_DATE with WINDOW_FREQUENCY

order_window = visit_w_date.copy()

order_window["ANCHOR_DATE"] = pd.to_datetime(order_window["ANCHOR_DATE"])
order_window["WINDOW_FREQUENCY"] = pd.to_numeric(order_window["WINDOW_FREQUENCY"], errors="coerce").astype("Int64")

wd_map = {'monday':0,'tuesday':1,'wednesday':2,'thursday':3,'friday':4,'saturday':5,'sunday':6}
order_window["__wd"] = order_window["WEEK_DAY_OF_ANCHOR_DATE"].str.lower().map(wd_map)

# end of each policy segment = next original ANCHOR_DATE (per customer), else +365d
order_window = order_window.sort_values(["CUSTOMER_ID","ANCHOR_DATE"]).reset_index(drop=True)
order_window["__seg_end"] = order_window.groupby("CUSTOMER_ID")["ANCHOR_DATE"].shift(-1)
order_window["__seg_end"] = order_window["__seg_end"].fillna(order_window["ANCHOR_DATE"] + pd.Timedelta(days=365))

# first aligned NEXT for each original row
cand = order_window["ANCHOR_DATE"] + pd.to_timedelta(order_window["WINDOW_FREQUENCY"].fillna(0), unit="D")
off  = (order_window["__wd"] - cand.dt.weekday) % 7
first_next = cand + pd.to_timedelta(off, unit="D")

# build ranges (use int(step) only when notna)
ranges = [
    pd.date_range(start=fn, end=se - pd.Timedelta(days=1), freq=f"{int(step)}D")
    if (pd.notna(fn) and pd.notna(step) and fn < se) else pd.DatetimeIndex([])
    for fn, se, step in zip(first_next, order_window["__seg_end"], order_window["WINDOW_FREQUENCY"])
]

# explode
out = order_window.loc[order_window.index.repeat([len(r) for r in ranges])].copy()
out["ANCHOR_DATE"] = pd.DatetimeIndex([d for r in ranges for d in r])

# compute the next aligned date for each expanded row
next_cand = out["ANCHOR_DATE"] + pd.to_timedelta(out["WINDOW_FREQUENCY"], unit="D")
next_off  = (out["__wd"] - next_cand.dt.weekday) % 7
out["NEXT_ANCHOR_SAME_WD"] = next_cand + pd.to_timedelta(next_off, unit="D")

# include original rows with their NEXT
orig = order_window.copy()
orig["NEXT_ANCHOR_SAME_WD"] = first_next

final = (
    pd.concat([orig, out], ignore_index=True)
      .drop(columns=["__wd","__seg_end"])
      .sort_values(["CUSTOMER_ID","ANCHOR_DATE"])
      .reset_index(drop=True)
)
order_window_final = final[(final['ANCHOR_DATE'] > pd.to_datetime('2024-06-05')) & 
       (final['ANCHOR_DATE'] < pd.to_datetime('2025-05-26'))]

In [ ]:
# Same customer but now each rows are windows of order
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(order_window_final[order_window_final['CUSTOMER_ID']==501738077])
len(order_window_final)

## Joining final table to orders data¶

In [ ]:

chunk_size = 1000
results = []

# Sort orders once for efficient tail(1)
orders_clean = orders_clean.sort_values(['CUSTOMER_ID', 'CREATED_DATE_EST'])

for start in range(0, len(final), chunk_size):
    end = start + chunk_size
    order_window_chunk = order_window_final.iloc[start:end]

    # Filter orders to only customers in this chunk
    cust_ids = order_window_chunk['CUSTOMER_ID'].unique()
    orders_subset = orders_clean[orders_clean['CUSTOMER_ID'].isin(cust_ids)]

    # Merge and filter by anchor window
    merged = pd.merge(order_window_chunk, orders_subset, on='CUSTOMER_ID', suffixes=('_final', '_order'))
    merged = merged[
        (merged['CREATED_DATE_EST'] >= merged['ANCHOR_DATE']) &
        (merged['CREATED_DATE_EST'] < merged['NEXT_ANCHOR_SAME_WD'])
    ]

    # Get most recent order per anchor window
    merged = merged.sort_values(['CUSTOMER_ID', 'ANCHOR_DATE', 'CREATED_DATE_EST'])
    recent = merged.groupby(['CUSTOMER_ID', 'ANCHOR_DATE'], as_index=False).tail(1)

    results.append(recent)

# Combine all chunks
recent_orders = pd.concat(results, ignore_index=True)

# Merge back to final
order_cols = ['CREATED_DATE_UTC', 'MATERIAL_ID', 'ORDER_QUANTITY', 'ORDER_TYPE', 'CREATED_DATE_EST']
final_with_orders = pd.merge(
    final,
    recent_orders[['CUSTOMER_ID', 'ANCHOR_DATE'] + order_cols],
    on=['CUSTOMER_ID', 'ANCHOR_DATE'],
    how='left'
)

final_with_orders['ORDER_EXISTS'] = final_with_orders['CREATED_DATE_EST'].notnull().astype(int)

final_with_orders.head()

## Fixing timezones in GA data

In [ ]:
from datetime import timedelta

# Extract state from SALES_OFFICE_DESCRIPTION (format: "City, ST")
final_with_orders["SALES_OFFICE_STATE"] = (
    final_with_orders['SALES_OFFICE_DESCRIPTION']
    .str.split(',')
    .str[1]
    .str.strip()
)

# Function for rough timezone offset (EST → local)
def est_to_local(event_time, state):
    offsets = {
        'AZ': -2,  # MST no DST from EST
        'WA': -3, 'OR': -3, 'CA': -3,  # PDT
        'CO': -2, 'NM': -2,  # MDT
        'ID': -3,  # PDT approx
        'NV': -3,  # PDT
        'UT': -2,  # MDT
        'NE': -2, 'WY': -2,  # MDT approx
        # Add others as needed
    }
    offset_hours = offsets.get(state, -3)  # Default -3
    return event_time + timedelta(hours=offset_hours)

# ✅ Use final_with_orders since that’s where the column exists
customer_state_map = (
    final_with_orders[['CUSTOMER_ID', 'SALES_OFFICE_STATE']]
    .drop_duplicates()
    .set_index('CUSTOMER_ID')['SALES_OFFICE_STATE']
)

# Map state to GA data
ga_clean['SALES_OFFICE_STATE'] = ga_clean['CUSTOMER_ID'].map(customer_state_map)

# Convert and apply time shift
ga_clean['EVENT_TIMESTAMP'] = pd.to_datetime(ga_clean['EVENT_TIMESTAMP'])
ga_clean['EVENT_TIMESTAMP_LOCAL'] = ga_clean.apply(
    lambda row: est_to_local(row['EVENT_TIMESTAMP'], row['SALES_OFFICE_STATE']),
    axis=1
)

print(ga_clean.head())


## Joining with GA data

In [ ]:
chunk_size = 1000
ga_chunks = []

# Sort GA events for consistency
ga_events = ga_clean.sort_values(['CUSTOMER_ID', 'EVENT_DATE'])

for start in range(0, len(final), chunk_size):
    end = start + chunk_size
    final_chunk = final_with_orders.iloc[start:end]

    # Filter GA events to only customers in this chunk
    cust_ids = final_chunk['CUSTOMER_ID'].unique()
    ga_subset = ga_events[ga_events['CUSTOMER_ID'].isin(cust_ids)]

    # Merge and filter by anchor window
    merged = pd.merge(final_chunk, ga_subset, on='CUSTOMER_ID', suffixes=('_final', '_event'))
    merged = merged[
        (merged['EVENT_DATE'] >= merged['ANCHOR_DATE']) &
        (merged['EVENT_DATE'] < merged['NEXT_ANCHOR_SAME_WD'])
    ]

    ga_chunks.append(merged)

# Combine all matching events
events_in_window = pd.concat(ga_chunks, ignore_index=True)



## Defining an Abandoned Cart and creating our final table¶

In [ ]:
#Aggregate GA data at cart level
cart_level_df = (
    events_in_window
    .assign(in_window=lambda df:
        (df["EVENT_DATE"] >= df["ANCHOR_DATE"]) &
        (df["EVENT_DATE"] < df["NEXT_ANCHOR_SAME_WD"]))
    .query("in_window")
    .groupby(["CUSTOMER_ID", "ANCHOR_DATE", "NEXT_ANCHOR_SAME_WD"], as_index=False)
    .agg(
        added_items=("EVENT_NAME", lambda x: (x == "add_to_cart").sum()),
        num_items_added=("ITEMS", lambda x: sum(
            len(items) for items in x[x.index[x.index.map(lambda i: events_in_window.loc[i, "EVENT_NAME"] == "add_to_cart")]]
        )),
        purchases=("EVENT_NAME", lambda x: (x == "purchase").sum()),
        button_clicks=("EVENT_NAME", lambda x: (x == "button_click").sum()),
        total_events=("EVENT_NAME", "count")
    )
    .assign(
        ABANDONED_CART=lambda df: ((df["added_items"] > 0) & (df["purchases"] == 0)).astype(int)
    )
)
cart_level_df.head()



In [ ]:
# Filter to carts with added items
cart_level_df = cart_level_df.query("added_items > 0").copy()
# abandoned cart rate
cart_level_df['ABANDONED_CART'].mean()

Our abandoned Cart Rate is about 15.15%!

In [ ]:
#Join back to final orders table
final_with_ga = final_with_orders.merge(
    cart_level_df,
    on=["CUSTOMER_ID", "ANCHOR_DATE", "NEXT_ANCHOR_SAME_WD"],
    how="inner"  # only keep rows where GA data exists
)

In [ ]:
# export the final_orders_ga
final_with_ga.to_csv('final_with_ga.csv', index=False)

# Plots

In [ ]:
abandon = pd.read_csv("final_with_ga.csv")

In [ ]:
# What is the abandon rate
print(f"The cart abandon rate is {abandon['ABANDONED_CART'].mean() * 100:.2f}%")

sns.countplot(x = abandon['ABANDONED_CART'])
plt.title('The distribution of abandon cart and not')

In [ ]:
# Which day of the week do customes tend to set their anchor date in
sns.countplot(x='WEEK_DAY_OF_ANCHOR_DATE', data=abandon)
plt.show()

In [ ]:
# Day of the week customer usually purchase 
# Convert the 'CREATED_DATE_UTC' column to datetime if it isn't already
abandon['CREATED_DATE_UTC'] = pd.to_datetime(abandon['CREATED_DATE_UTC'])

# Extract day of the week (Monday=0, Sunday=6)
abandon['Day_of_week_last_event'] = abandon['CREATED_DATE_UTC'].dt.day_name()

# Plot the counts of each day
ax = sns.countplot(x='Day_of_week_last_event', data=abandon[abandon['ABANDONED_CART']==1], order=[
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Count of Created Dates by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Count')
plt.show()

Mondays and earlier weekdays seem to be the days when businesses tend to prep their stocks

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(
    x='SHIPPING_CONDITIONS_DESC', 
    y='ABANDONED_CART', 
    data=abandon, 
    estimator='mean', 
    errorbar=None, 
    palette='coolwarm'
)
plt.title('Abandonment Rate by Shipping Condition')
plt.xlabel('Shipping Condition')
plt.ylabel('Average Abandonment Rate')
plt.show()

In [ ]:
plt.figure(figsize=(9,5))
sns.barplot(
    x='DISTRIBUTION_MODE_DESC', 
    y='ABANDONED_CART', 
    data=abandon, 
    estimator='mean', 
    palette='viridis'
)
plt.title('Abandonment Rate by Distribution Mode')
plt.xticks(rotation=45)
plt.ylabel('Average Abandonment Rate')
plt.xlabel('Distribution Mode')
plt.show()

In [ ]:
abandon_customer = pd.merge(abandon, customer_clean, on='CUSTOMER_ID', how='left')

sns.countplot(x='COLD_DRINK_CHANNEL_DESCRIPTION',hue='ABANDONED_CART', data=abandon_customer)
plt.xticks(rotation=90)
plt.show()

## Results & Findings

This section summarizes the key patterns and insights derived from the exploratory data analysis conducted across multiple datasets — including orders, visits, customer profiles, sales, and Google Analytics. The goal was to identify behavioral and operational factors contributing to **cart abandonment**.

---

### Overall Cart Abandonment Rate
The overall **cart abandonment rate is approximately 15.1%**, indicating that a considerable proportion of customers initiate orders but do not complete them.  
While a majority of users proceed to purchase, this 15% gap represents a strong opportunity area for optimization through better engagement and faster fulfillment.

---

### Day-of-Week Patterns
- Customers tend to **set their anchor dates** and place orders more frequently on **Mondays and Wednesdays**.  
- Abandoned carts are **least common on weekends**, suggesting that business buyers typically place and finalize orders early in the workweek when stock planning occurs.

**Interpretation:**  
Most activity aligns with B2B restocking cycles. Operational outreach or targeted reminders early in the week could help capture these high-intent windows.

---

### Cold Drink Channel Analysis
- The **Restaurant** and **Distributor** segments generate the largest order volume, but also represent a significant share of total abandonment.  
- The **Hot Beverage** channel shows the **highest abandonment percentage (around 27%)**, indicating irregular ordering or uncertain purchase intent.  
- Channels like **Workplace** and **Clinic** maintain the lowest abandonment rates (~8–12%), reflecting more stable and predictable ordering behaviors.

**Interpretation:**  
Abandonment is most prevalent in categories where order timing is more flexible or optional (e.g., cafés and small distributors). Strengthening automated reordering or replenishment tools could improve conversion in these segments.

---

### Abandonment Rate by Shipping Condition
- Customers facing **longer shipping windows (72 Hours)** show the **highest abandonment rate (~22%)**.  
- **Faster delivery options (24 Hours and Dropsite 48 Hours)** have the **lowest abandonment rates (~10–15%)**.

**Interpretation:**  
Shipping speed plays a major role in purchase completion. Faster fulfillment likely boosts customer confidence and urgency to finalize orders.  
Offering express options or clear delivery-time visibility could reduce abandonment in slower shipping categories.

---

### Abandonment Rate by Distribution Mode
- The **‘Tell Sell’** distribution mode shows the **highest abandonment (~30%)**, likely due to manual order handling and slower confirmation processes.  
- More automated and efficient modes such as **Full Service**, **OFS**, and **Rapid Delivery** have much lower abandonment rates (~12–16%).  
- **Bulk Distribution** also records moderate drop-offs, potentially due to larger, more complex orders.

**Interpretation:**  
Order fulfillment channels with greater automation and transparency tend to reduce customer hesitation.  
Digitizing manual processes (e.g., ‘Tell Sell’) could yield immediate improvement in conversion rates.

---

### Abandonment by Visit Frequency
- Customers with **lower visit frequency (monthly or infrequent buyers)** exhibit **higher abandonment**.  
- Regular buyers, particularly those visiting weekly or biweekly, show **stronger follow-through rates**.

**Interpretation:**  
Engagement frequency strongly correlates with completion. Encouraging repeat visits through loyalty programs, personalized prompts, or scheduled order reminders could improve retention and reduce cart abandonment.

---

### Abandonment Rate by Sales Office
- Offices such as **Richfield, UT**, **Price, UT**, and **Scottsbluff, NE** show the **highest abandonment rates (above 20%)**.  
- Conversely, **Boise, ID**, **Ogden, UT**, and **Draper, UT** maintain **the lowest rates (below 10%)**, signaling stronger engagement and operational efficiency.

**Interpretation:**  
Regional disparities in performance may stem from differences in local order management, fulfillment reliability, or customer communication.  
Prioritizing high-abandonment offices for sales training or process optimization could lead to significant improvement in overall conversion rates.

---

## Summary of Key Insights

| Factor | Observation | Business Implication |
|--------|--------------|----------------------|
| **Overall Abandonment** | 15.1% of carts are left incomplete | Clear opportunity for process and UX optimization |
| **Day of Week** | Orders peak on Monday & Wednesday | Focus customer engagement early in the week |
| **Shipping Condition** | Higher abandonment with longer delivery windows | Offer faster or transparent shipping options |
| **Distribution Mode** | Manual “Tell Sell” has the highest abandonment | Automate or streamline manual order channels |
| **Customer Frequency** | Regular visitors complete more purchases | Incentivize frequent ordering behavior |
| **Regional Offices** | Large performance gaps between offices | Targeted interventions in high-abandonment regions |

---

**Overall Conclusion:**  
Cart abandonment in this dataset appears to be driven by a mix of **logistical factors (delivery time, fulfillment type)** and **behavioral factors (visit frequency, timing)**.  
By focusing on faster delivery, automation in manual processes, and improved engagement for infrequent buyers, the organization can meaningfully reduce its abandonment rate and improve sales efficiency.

## Group Member Contribution

This project was completed as part of the **IS 6480: Capstone Project** course at the **University of Utah**, under the supervision of **Professor Jeff Webb** and **Swire Coca-Cola** as the industry partner.

Our team collaborated closely throughout the project lifecycle — from data understanding and cleaning to modeling, analysis, and final presentation.  
Below is a summary of each member’s key contributions:

| **Team Member** | **Primary Contributions** |
|------------------|---------------------------|
| **Finlay Dunn** | Led exploratory data analysis (EDA), integrated Google Analytics (GA) event logs with Orders and Visit Plan datasets, defined the logic for order window construction, and calculated cart abandonment metrics.  |
| **Huong** | Processed and merged the Visit Plan, and Cutoff Time datasets to create the master table. Contributed to data cleaning, frequency calculations, and initial GA event analysis. |
| **Sudeeptha** | Focused on data validation and consistency checks across tables (Orders, Sales, GA). Compiled the notebook and assisted in visual summaries of order patterns and cart activity. |
| **Shawal Fida** | Assisted in constructing descriptive statistics and visual summaries of order patterns and cart activity, contributed to data validation, and worked on interpreting the results and findings. 
